<p style="font-weight:bold;"> <span style="font-size: 36px"> Interact With Import Scope Calculation</span> </p>

<br><br>
After successfully loading the parsed data in the dataSource, the import scopes with for a specific partition can be re-evaluated in this notebook. 
The import scopes can then be displayed, examined and used for further calculations.

# Basic Initialization

In [0]:
#!import "../../Constants/CalculationEngine"

In [0]:
var mystring = $"#!eval-notebook \"//{projectName}/{environmentName}/{notebookName}\"";
mystring

In [0]:
#!eval mystring

In [0]:
#!eval-notebook "../../Initialization/InitSystemorphBaseToMemory"
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.InitializeFrom(DataSource);

# Import of parsed data

In [0]:
Debug.Enable = true;

In [0]:
await Import.FromFile("../../Files/TransactionalData/NominalCashflows_CH_2020_12.csv")
            .WithFormat(ImportFormats.Cashflow)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync()

# Args

In [0]:
var reportingNode = "CH";
var year = 2020;
var month = 12;
var importFormat = ImportFormats.Cashflow;
var scenario = (string)null;

ImportArgs args = new ImportArgs(reportingNode, year, month, default(Periodicity), scenario, importFormat);
args

# Construct ImportStorage

In [0]:
var storage = new ImportStorage(args, DataSource, Workspace);
await storage.InitializeAsync();

# Create universe and Identities

In [0]:
var universe = Scopes.ForStorage(storage).ToScope<IModel>();
var identities = universe.GetScopes<GetIdentities>(storage.DataNodesByImportScope[ImportScope.Primary]).SelectMany(s => s.Identities);

In [0]:
var ret = universe.GetScopes<CumulatedNominal>(identities)
                .SelectMany(x => x.PresentValues)
                .Select(x => new {Value= x.Value, 
                                  Id = x.Identity.Id, 
                                  AmoutType = x.Identity.AmountType,
                                  EstimateType = x.Identity.EstimateType,
                                  AccidentYear = x.Identity.AccidentYear});


In [0]:
ret

In [0]:
var id = identities.Skip(1).First();
id

In [0]:
Scopes.ForIdentities(id.RepeatOnce(), storage).ToScopes<CumulatedNominal>().Single().PresentValues.First()